In [68]:
# Enable module reloading
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

import datetime
import os
import sys
sys.path.append('..')

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import time;

from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, concatenate
from tensorflow.keras.utils import to_categorical
from src.data.load_data import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [108]:
df = load_dataset(filename='../data/Jester-Dataset-ratings.csv')
user_ids, joke_ids, ratings = get_data(df, batch_size=20000)
# batch_size = 20000
        
# matrix = df[0:batch_size].pivot(index='USER_ID', columns='JOKE_ID', values='Rating').fillna(0)
# user_ids = matrix.index.tolist()
# joke_ids = matrix.columns.tolist()
# ratings = matrix.values.flatten().tolist()
# ratings = np.array(cur_df.pivot(index='USER_ID', columns='JOKE_ID', values='Rating').fillna(0)).tolist()

In [109]:
user_onehot = to_categorical(user_ids)
joke_onehot = to_categorical(joke_ids)

In [110]:
user_onehot

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [111]:
final_vector = []
for u, i in zip(user_onehot, joke_onehot):
    final_vector.append(np.concatenate((u, i), axis=None))

In [112]:
np.shape(final_vector)

(20000, 662)

In [113]:
np.shape(ratings)

(20000,)

In [126]:
from src.models.model import JokeRecommender

model = JokeRecommender(final_vector)

model.compile(
    optimizer='adam', 
    loss='mean_squared_error', 
    metrics=['accuracy']
)

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=os.path.join("../logs", str(time.time())),
    histogram_freq=1)

model.fit(
    x=np.array(final_vector),
    y=np.array(ratings),
    batch_size=100, 
    epochs=50,
    callbacks=[tensorboard_callback],
    validation_split=0.2
)

Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 5s 283us/sample - loss: 34.3254 - accuracy: 0.0017 - val_loss: 33.4773 - val_accuracy: 0.0000e+00
Epoch 2/50
16000/16000 [==============================] - 3s 209us/sample - loss: 33.4340 - accuracy: 0.0000e+00 - val_loss: 33.0640 - val_accuracy: 0.0000e+00
Epoch 3/50
16000/16000 [==============================] - 3s 211us/sample - loss: 33.1885 - accuracy: 0.2956 - val_loss: 33.0603 - val_accuracy: 0.1922
Epoch 4/50
16000/16000 [==============================] - 3s 211us/sample - loss: 33.0916 - accuracy: 0.5592 - val_loss: 33.0603 - val_accuracy: 1.0000
Epoch 5/50
16000/16000 [==============================] - 3s 210us/sample - loss: 33.0729 - accuracy: 0.6272 - val_loss: 33.0603 - val_accuracy: 1.0000
Epoch 6/50
15600/16000 [============================>.] - ETA: 0s - loss: 33.1086 - accuracy: 0.6197

KeyboardInterrupt: 